<a href="https://colab.research.google.com/github/suinkangme/comp433_project/blob/main/COMP433_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Developing a robust CNN model to address the challenge of learning with label noise in  CIFAR10 dataset

- CIFAR10 Label : ‘airplane’, ‘automobile’, ‘bird’, ‘cat’, ‘deer’, ‘dog’, ‘frog’, ‘horse’, ‘ship’, ‘truck’.

- image size : 3x32x32




In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np


## Load and normalize CIFAR10

- Transform them to Tensors of normalized range [-1, 1].

In [2]:
transform = transforms.Compose(
                              [transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [3]:
train_data = torchvision.datasets.CIFAR10(root='./data',
                                          train=True,
                                          download=True,
                                          transform = transform)

test_data = torchvision.datasets.CIFAR10(root='./data',
                                          train=False,
                                          download=True,
                                          transform = transform)

100%|██████████| 170498071/170498071 [00:03<00:00, 45144453.72it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
train_loader = torch.utils.data.DataLoader(train_data,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=2)

test_loader = torch.utils.data.DataLoader(test_data,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          num_workers=2)

## Define a CNN model - baseline

In [5]:
class BaselineModel(nn.Module):
  def __init__(self):
    super(BaselineModel,self).__init__()
    self.features = nn.Sequential(
      nn.Conv2d(3, 8, kernel_size=3, padding = 1),  # (input channel, output channels, kernel size, padding)  32*32*8
      nn.ReLU(inplace=True), # activation function modifies the input tensor directly
      nn.Conv2d(8, 16, kernel_size=3, padding=1),
      nn.ReLU(inplace=True),
      nn.MaxPool2d(kernel_size=2,stride=2), # 16*16*16

      nn.Conv2d(16, 32, kernel_size=3, padding=1),
      nn.ReLU(inplace=True),
      nn.Conv2d(32, 64, kernel_size=3, padding=1),
      nn.ReLU(inplace=True),
      nn.Conv2d(64, 128, kernel_size=3, padding=1),
      nn.MaxPool2d(kernel_size=2,stride=2) # 8*8*128
    )

    # fully connected layers
    self.fc_layers = nn.Sequential(
      nn.Linear(128*8*8, 120),
      nn.Linear(120,84),
      nn.Linear(84,10)
    )


  def forward(self, x):
    x = self.features(x)
    x = torch.flatten(x,1)
    x = self.fc_layers(x)
    return x

## Noise Labeling
- 5 different noise levels (10%,
30%, 50%, 80%, 90%)

### Symmetric label noise

In [6]:
def add_symmetric_noise(labels, epsilon):
    num_labels = len(labels)
    num_flips = int(epsilon * num_labels)

    # choose the label to be flipped
    flip_indices = np.random.choice(num_labels, num_flips, replace=False)

    # filp the label
    labels[flip_indices] = np.random.randint(0, 10, num_flips)

    return labels

### Asymmetric label noise

## Train the model - baseline

### Train with the symmetric noise labeling

In [8]:
# add noisy label in training data
noise_levels = [0.1, 0.3, 0.5, 0.8, 0.9]

for epsilon in noise_levels:
    for batch in train_loader:
        inputs, labels = batch
        labels_noisy = torch.from_numpy(add_symmetric_noise(labels.numpy(), epsilon))
        break

In [9]:
# Training on GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

net = BaselineModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001)

In [11]:
net = net.to(device)
net.train()
num_epochs = 5

for epsilon in noise_levels:
    print(f"Training with noise level: {epsilon}")

    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            # Add symmetric noise to labels
            labels_noisy = torch.from_numpy(add_symmetric_noise(labels.numpy(), epsilon))

            # Move data to GPU
            inputs, labels_noisy = inputs.to(device), labels_noisy.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = net(inputs)
            loss = criterion(outputs, labels_noisy)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

Training with noise level: 0.1
Epoch 1/5, Loss: 2.313051700592041
Epoch 2/5, Loss: 1.8369474411010742
Epoch 3/5, Loss: 1.708470344543457
Epoch 4/5, Loss: 0.8492963910102844
Epoch 5/5, Loss: 1.815744161605835
Training with noise level: 0.3
Epoch 1/5, Loss: 2.0510189533233643
Epoch 2/5, Loss: 1.7332820892333984
Epoch 3/5, Loss: 2.009709358215332
Epoch 4/5, Loss: 1.8218185901641846
Epoch 5/5, Loss: 2.135478973388672
Training with noise level: 0.5
Epoch 1/5, Loss: 1.856129765510559
Epoch 2/5, Loss: 2.4521522521972656
Epoch 3/5, Loss: 1.8051732778549194
Epoch 4/5, Loss: 1.986630916595459
Epoch 5/5, Loss: 1.9094387292861938
Training with noise level: 0.8
Epoch 1/5, Loss: 2.222879409790039
Epoch 2/5, Loss: 1.9656038284301758
Epoch 3/5, Loss: 2.0789084434509277
Epoch 4/5, Loss: 2.2096428871154785
Epoch 5/5, Loss: 2.5367319583892822
Training with noise level: 0.9
Epoch 1/5, Loss: 2.3774466514587402
Epoch 2/5, Loss: 1.8959318399429321
Epoch 3/5, Loss: 2.524543285369873
Epoch 4/5, Loss: 2.2416779

### Save the trained model

In [ ]:
# save the baseline model
PATH = './BaselineModel.pth'
torch.save(net.state_dict(), PATH)

## Testing